# Example `KBackground.Estimator` workflow with Kepler TPFs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
from kbackground import Estimator
import astropy.units as u

First we are going to get a number of the TPFs in a large superstamp in campaign 8. This will let us see a patch of data with a rolling band.

In [3]:
tpfs = lk.search_targetpixelfile('KIC 8557374', quarter=5, cadence='long', radius=10 * u.arcmin).download_all()

Next we will extract the row, column and flux values from all the TPFs.

In [4]:
rows, columns, fluxs, cadencenos = [], [], [], []
for tpf in tpfs:
    column, row = np.meshgrid(np.arange(tpf.column, tpf.shape[2] + tpf.column), np.arange(tpf.row, tpf.shape[1] + tpf.row))
    aper = np.ones(tpf.shape[1:], bool)
    rows.append(row[aper])
    columns.append(column[aper])
    fluxs.append(tpf.flux.value[:, aper])
    cadencenos.append(tpf.cadenceno)

Next we can build the `Estimator` object, passing in the row, column, and flux values from all the TPFs.

In [5]:
r, c, f = np.hstack(rows), np.hstack(columns), np.hstack(fluxs)
k = np.isfinite(f).all(axis=0)

In [ ]:
e = Estimator(cadencenos[0], r[k], c[k], f[:, k], 15, 30)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].pcolormesh(cadencenos[0], e.unq_row, e.bf, vmin=-10, vmax=10)
ax[0].set(xlabel='Cadence Number', ylabel='Row Number', title='Data')
ax[1].pcolormesh(cadencenos[0], e.unq_row, e._model, vmin=-10, vmax=10)
ax[0].set(xlabel='Cadence Number', ylabel='Row Number', title='Model')

In [ ]:
t = tpf.time.value

In [ ]:
tdx = 890
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].scatter(c[k], r[k], s=5, c=e.flux[tdx] - np.median(e.flux, axis=0)[None, :], vmin=-10, vmax=10, cmap='coolwarm')
ax[0].set_aspect('equal')
ax[0].set(title='Data', xlabel='Column', ylabel='Row')
ax[1].scatter(c[k], r[k], s=5, c=e.model[tdx], vmin=-10, vmax=10, cmap='coolwarm')
ax[1].set_aspect('equal')
ax[1].set(title='Model', xlabel='Column', ylabel='Row')

In [ ]:
plt.plot(tpf.time.value, e.flux[:, 900], label='Data', c='grey')
plt.plot(tpf.time.value, e.flux[:, 900] - e.model[:, 300], label='Data', c='k')
plt.plot(tpf.time.value, e.model[:, 900], label='Model', c='r')
plt.legend()
plt.xlabel('Time [BKJD]')
plt.ylabel('Flux [counts]')